How to authenticate to Google APIs?



How to exchange token for oauth client?

google oauth token client?


In [4]:
var readline = require('readline');
var process = require('process');
var path = require('path');
var fs = require('fs');
var util = require('util');
var {OAuth2Client} = require('google-auth-library');

var importer = require('../Core');
//var runSeleniumCell = importer.import('selenium cell');

// Load client secrets from a local file.
var SCOPES = ['https://www.googleapis.com/auth/calendar.readonly'];
var SECRET_PATH, credentials;
var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '', '.credentials');

try {
    SECRET_PATH = path.join(TOKEN_DIR, 'client_secret.json');
    credentials = JSON.parse(fs.readFileSync(SECRET_PATH).toString());
} catch(e) {
    console.log(e);
}

function storeToken(token, tokenPath) {
    fs.writeFileSync(tokenPath, JSON.stringify(token, null, 4));
    console.log('Token stored to ' + tokenPath);
}

function receiveCode(code, oauth2Client, tokenPath) {
    return util.promisify(oauth2Client.getToken.bind(oauth2Client))(code)
        .then(token => {
            console.log('recieved token: ' + token.access_token);
            oauth2Client.setCredentials(token);
            storeToken(token, tokenPath);
            return oauth2Client;
        })
}

async function errorFallback(up, authUrl, oauth2Client, tokenPath) {
    if(up.message.includes('ECONNREFUSED')
      || up.message.includes('find module')
      || up.message.includes('runSeleniumCell is not defined')) {
        console.log('can\'t authenticate with selenium, waiting for user input.');
        console.log('Authorize this app by visiting this url:', authUrl);
        //if (process.stdout.isTTY) {
        //}
        var interface = readline.createInterface(process.stdin, process.stdout);
        var code = await new Promise(resolve => interface.question(
            'Enter the code from that page here: ', resolve))
        interface.close();
        return receiveCode(code, oauth2Client, tokenPath)
    } else {
        throw up;
    }
}

async function renewToken(oauth2Client, scopes, tokenPath) {
    var authUrl = oauth2Client.generateAuthUrl({
        access_type: 'offline',
        scope: scopes
    })
    try {
        var {authorizeSelenium} = await runSeleniumCell(['log in Google', 'selenium authorize google access'])
        console.log('using selenium to authorize: ' + authUrl)
        var code = await authorizeSelenium(authUrl) 
        return receiveCode(code, oauth2Client, tokenPath)
    } catch (up) {
        return errorFallback(up, authUrl, oauth2Client, tokenPath)
    }
}

async function authorize(scopes = SCOPES) {
    const tokenPath = path.join(TOKEN_DIR, scopes.join('')
        .replace(/[^a-z]+/ig, '_') + '.json')
    var oauth2Client = new OAuth2Client(
        credentials.web.client_id,
        credentials.web.client_secret,
        credentials.web.redirect_uris[0])

    try {
        // Check if we have previously stored a token.
        oauth2Client.setCredentials(JSON.parse(fs.readFileSync(tokenPath)));
        await oauth2Client.getAccessToken()
        return oauth2Client
    } catch (up) {
        // if the token file isn't found start a new auth
        if(up.message == 'invalid_token'
          || up.code === 'ENOENT') {
            return renewToken(oauth2Client, scopes, tokenPath);
        } else {
            throw up;
        }
    }
}

module.exports = authorize;


[Function: authorize]

How to use selenium to authorize Google access?



In [6]:
var loginGoogle = importer.import('log in google');

function authorizeSelenium(authUrl) {
    return client
        .url(authUrl)
        .then(() => loginGoogle(client))
        .then(() => client.$('#submit_approve_access'))
        .then(el => el.waitForDisplayed(3000))
        .then(() => client.$('#submit_approve_access'))
        .then(el => el.moveTo())
        .then(() => client.$('#submit_approve_access content'))
        .then(el => el.waitForDisplayed(3000))
        .then(() => client.$('#submit_approve_access content'))
        .then(el => el.click())
        .then(() => client.$('textarea'))
        .then(el => el.waitForDisplayed(4000).then(() => el.getValue()))
};

module.exports = authorizeSelenium;


ReferenceError: client is not defined

In [11]:
var util = require('util');
var {OAuth2Client} = require('google-auth-library');
var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '', '.credentials');
var SECRET_PATH = path.join(TOKEN_DIR, 'client_secret.json');
var credentials = JSON.parse(fs.readFileSync(SECRET_PATH).toString());

var oauth2Client = new OAuth2Client(
    credentials.installed.client_id,
    credentials.installed.client_secret,
    credentials.installed.redirect_uris[0]);

$$.async()
util.promisify(oauth2Client.getToken.bind(oauth2Client))
('4/QwGy62FM_KlubQwYvKLAu5z1xrT0cF85Zwb_3U7-IHvmm1gJZ-hj0Qo')
    .then(r => $$.sendResult(JSON.stringify(r)))
    .catch(e => $$.sendError(e))



'{"access_token":"ya29.GlsCB84KBjXZUV9jPaoIhsz5HJDY1iTYU759khvyfPTZBhWIYiEPAhAfZj9WFZpqT3CmJW048vhD6IY7PeGZyDdZSMp6gnuuj_IEgdLwrPreWtPcBIoja9aoVQYa","refresh_token":"1/jWT3jd7TP3Nr7c8_ggew4ZU85_YOeNsOKOaIYl_Miu4","scope":"https://www.googleapis.com/auth/calendar","token_type":"Bearer","expiry_date":1557191028718}'